In [1]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy.linalg import inv
import math

In [2]:
# Two telecommunication companies, two tech companies, and Walmart
expected_return = []
tickers = ['CMCSA','T', 'AAPL', 'MSFT', 'WMT']


start_date = '2014-01-01'
end_date = '2014-12-31'
for ticker in tickers:
    unit_cost = data.DataReader(ticker, 'yahoo', start_date)['Open'][0]
    adj_close = data.DataReader(ticker, 'yahoo', end_date)['Adj Close'][0]
    ticker_return = ((adj_close - unit_cost) / unit_cost)*100
    expected_return.append(ticker_return)
    
start_date = '2015-01-01'
end_date = '2015-12-31'
for ticker in tickers:
    unit_cost = data.DataReader(ticker, 'yahoo', start_date)['Open'][0]
    adj_close = data.DataReader(ticker, 'yahoo', end_date)['Adj Close'][0]
    ticker_return =  ((adj_close - unit_cost) / unit_cost)*100
    expected_return.append(ticker_return)
    
start_date = '2016-01-01'
end_date = '2016-12-30'
for ticker in tickers:
    unit_cost = data.DataReader(ticker, 'yahoo', start_date)['Open'][0]
    adj_close = data.DataReader(ticker, 'yahoo', end_date)['Adj Close'][0]
    ticker_return =  ((adj_close - unit_cost) / unit_cost)*100
    expected_return.append(ticker_return)
    
    
start_date = '2017-01-01'
end_date = '2017-12-29'
for ticker in tickers:
    unit_cost = data.DataReader(ticker, 'yahoo', start_date)['Open'][0]
    adj_close = data.DataReader(ticker, 'yahoo', end_date)['Adj Close'][0]
    ticker_return =  ((adj_close - unit_cost) / unit_cost)*100
    expected_return.append(ticker_return)
    
start_date = '2018-01-01'
end_date = '2018-12-31'
for ticker in tickers:
    unit_cost = data.DataReader(ticker, 'yahoo', start_date)['Open'][0]
    adj_close = data.DataReader(ticker, 'yahoo', end_date)['Adj Close'][0]
    ticker_return =  ((adj_close - unit_cost) / unit_cost)*100
    expected_return.append(ticker_return)

In [3]:
expected_return = np.array(expected_return)
years = [];
for i in range(2014, 2019):
    years.append(i)

returns_df = pd.DataFrame(expected_return.reshape(5,5), columns = tickers)
returns_df['year'] = years
display(returns_df)

,CMCSA,T,AAPL,MSFT,WMT,year
0,1.705207,-28.388810,27.456902,11.344808,-3.731348,2014
1,-10.146865,-18.620589,-11.904417,9.353252,-35.554956,2015
2,18.108101,4.171407,7.569124,8.095585,6.627371,2016
3,9.813918,-20.169875,41.511218,31.603032,36.547809,2017
4,-18.278692,-32.165617,-8.887455,15.891587,-8.055473,2018


In [4]:
# Read in all datasets
df2018 = pd.read_csv("2018_Financial_Data.csv")
df2017 = pd.read_csv("2017_Financial_Data.csv")
df2016 = pd.read_csv("2016_Financial_Data.csv")
df2015 = pd.read_csv("2015_Financial_Data.csv")
df2014 = pd.read_csv("2014_Financial_Data.csv")

# Insert data into list for iteration purposes
list1 = [df2018, df2017, df2016, df2015, df2014]

# This loop retrieves 5 years of data for each of the selected stocks so that the expected return value (which is
# calculated above) can be appended to each stock.
for i in range(5):
    
    # Modify data to get desired amount of features
    financial_data = pd.DataFrame(list1[i], columns = ['Symbol', 'Revenue', 'Revenue Growth', 'Gross Profit', 
                                                  'Operating Income', 'Earnings before Tax', 'Free Cash Flow', 
                                                  'Net Income', 'Total current assets',
                                                  'Operating Expenses',  'Net Debt', 'Short-term debt', 'Long-term debt', 
                                                  'Total shareholders equity', 'Weighted Average Shs Out', 
                                                  'Total current liabilities', 'Total debt', 'Total liabilities']) 
    nan_value = float("NaN")
    target_data = financial_data

    # Get data from each selected stock
    s1 = pd.DataFrame(financial_data.loc[financial_data.loc[:,('Symbol')] == "CMCSA", :])
    s2 = pd.DataFrame(financial_data.loc[financial_data['Symbol'] == "T", :])
    s3 = pd.DataFrame(financial_data.loc[financial_data['Symbol'] == "AAPL", :])
    s4 = pd.DataFrame(financial_data.loc[financial_data['Symbol'] == "MSFT", :])
    s5 = pd.DataFrame(financial_data.loc[financial_data['Symbol'] == "WMT", :])
    
    # Append stock data together throughout 5 years
    if i == 0:
        s1_total = s1
        s2_total = s2
        s3_total = s3
        s4_total = s4
        s5_total = s5
    else:
        s1_total = s1_total.append(s1)
        s2_total = s2_total.append(s2)
        s3_total = s3_total.append(s3)
        s4_total = s4_total.append(s4)
        s5_total = s5_total.append(s5)

# Add on Expected Return column to each stock

s1_total['Expected Return'] = [expected_return[0],expected_return[5], expected_return[10], 
                            expected_return[15], expected_return[20]]
s2_total['Expected Return'] = [expected_return[1],expected_return[6], expected_return[11], 
                             expected_return[16], expected_return[21]]
s3_total['Expected Return'] = [expected_return[2],expected_return[7], expected_return[12], 
                             expected_return[17], expected_return[22]]
s4_total['Expected Return'] = [expected_return[3],expected_return[8], expected_return[13], 
                             expected_return[18], expected_return[23]]
s5_total['Expected Return'] = [expected_return[4],expected_return[9], expected_return[14], 
                             expected_return[19], expected_return[24]]

In [5]:
# Create an array of the tables above
dataTables = [s1_total, s2_total, s3_total, s4_total, s5_total]

# Loop through each of the tables and calculate the pararmeters for multiple linear regression 
# After calculating the parameters, test the regression and calculate RMSE
for i in range(0, len(dataTables)):
    
    # Find what the X and Y matrices are for normal equation
    x = dataTables[i].drop('Symbol',1)
    x = x.drop('Expected Return', 1).to_numpy()
    y = dataTables[i]['Expected Return'].to_numpy()
    
    # Calculate the parameters
    XTX = x.transpose().dot(x)
    XTX_inverse = inv(np.matrix(XTX))
    XTX_inverse_X = XTX_inverse.dot(x.transpose())
    param_theta = XTX_inverse_X.dot(y)
    

    # Find RMSE for the model using the training data and targets we have
    error_sum = 0
    for j in range (0,5):
        y_pred = param_theta.dot(x[j])
        pred_difference = y_pred - returns_df[tickers[i]][j]
        error_sum = error_sum + ((pred_difference**2) / 5)
    RMSE = math.sqrt(error_sum)
    print("RMSE for ", tickers[i], " is ", RMSE) #print the RMSE

RMSE for  CMCSA  is  2.3027827207312885
RMSE for  T  is  105.51891347566765
RMSE for  AAPL  is  129.46772053770098
RMSE for  MSFT  is  437.5444310631503
RMSE for  WMT  is  15.728002433485424


In [6]:
def get_indexes_min_value(l):
    min_value = min(l)
    if l.count(min_value) > 1:
        return [i for i, x in enumerate(l) if x == min(l)]
    else:
        return l.index(min(l))
    
# Create an array of the tables above
dataTables = [s1_total, s2_total, s3_total, s4_total, s5_total]

# Loop through each of the tables and calculate the pararmeters for multiple linear regrression 
# After calculating the parameters, test the regression and calcualte RMSE
for i in range(0, len(dataTables)):
    
    # Find what are the X and Y matrices for normal equation
    x = dataTables[i].drop('Symbol',1)
    x = x.drop('Expected Return', 1).to_numpy()
    y = dataTables[i]['Expected Return'].to_numpy()
    
    # Calculate the parameters with regularization
    XTX = x.transpose().dot(x)
    a = np.eye(len(XTX), dtype=int)
    a[0,0] = 0
    lam = np.arange(0, 100, 0.001).tolist() #range of lambda to select the best one
    error_list = []
    
    # Loop that selects the parameters and lambda that gives the lowest error
    for l in range(0, len(lam)):
        XTX_reg = XTX + lam[l]*a
        XTX_inverse = inv(np.matrix(XTX_reg))
        XTX_inverse_X = XTX_inverse.dot(x.transpose())
        param_theta = XTX_inverse_X.dot(y)
       
        error_sum = 0
        for j in range (0,5):
            y_pred = param_theta.dot(x[j])
            pred_difference = y_pred - returns_df[tickers[i]][j]
            error_sum = error_sum + ((pred_difference**2) / 5)
        RMSE = math.sqrt(error_sum)
        error_list.append(RMSE)
    lowestRMSE = min(error_list)
    optimumLambda = lam[get_indexes_min_value(error_list)]
    print("RMSE for ", tickers[i], " is ", lowestRMSE, " with parameter lambda ", optimumLambda, " for regularization.") 
    
    # Show the parameters that optimize this
    XTX_reg = XTX + optimumLambda*a
    XTX_inverse = inv(np.matrix(XTX_reg))
    XTX_inverse_X = XTX_inverse.dot(x.transpose())
    param_theta = XTX_inverse_X.dot(y)
    
    var =[]
    for k in range (0,17):
        var.append("x"+ str(k))
    parameter = pd.DataFrame(param_theta, columns = var)
    display(parameter) #show the model that is optimum for each stock

RMSE for  CMCSA  is  0.29494011750314303  with parameter lambda  83.938  for regularization.


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16
0,-1954.610324,-13.588065,1954.610324,-1.311502e-08,3.949972e-08,-4.689088e-08,-1.886723e-09,-4.444366e-09,8.836250e-10,-1.442325e-09,-3.472263e-08,9.937977e-10,-1.294061e-10,-1.115542e-08,-8.623470e-09,8.080249e-10,1.631437e-09


RMSE for  T  is  5.22090216450456  with parameter lambda  53.262  for regularization.


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16
0,1.175710e-10,323.249085,1.058649e-08,1.605478e-08,-8.966058e-08,5.472063e-08,5.779072e-08,-3.129783e-08,7.028638e-10,-1.386602e-09,-1.483228e-08,-1.702145e-09,-3.302487e-09,1.181703e-07,8.398156e-09,8.539495e-09,-5.064354e-09


RMSE for  AAPL  is  2.0869565468397147  with parameter lambda  9.613  for regularization.


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16
0,6.173885e-11,-7.157945,4.366141e-09,-1.388950e-08,1.034883e-08,-1.827689e-10,4.433885e-09,-1.643368e-10,-1.274393e-08,-2.070309e-09,-4.530265e-09,-3.945685e-10,-9.678924e-10,-1.159231e-08,-2.511816e-10,2.569001e-11,1.469777e-09


RMSE for  MSFT  is  1.9787003636493965  with parameter lambda  60.76  for regularization.


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16
0,-9.278749e-09,-387.935939,1.940576e-08,-2.914326e-08,2.994523e-08,-1.018125e-08,5.546324e-10,4.895590e-10,1.105035e-09,1.843752e-09,1.112396e-09,3.343661e-11,3.924477e-11,4.769383e-09,-4.527067e-10,2.439446e-10,-7.207667e-10


RMSE for  WMT  is  0.13149039880689933  with parameter lambda  89.668  for regularization.


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16
0,5.224008e-10,-40.317588,-1.478222e-10,8.744766e-09,7.749380e-09,-1.447070e-08,-4.056107e-09,-7.769134e-10,1.781037e-09,-4.509392e-09,-1.061452e-08,-7.009407e-09,6.010268e-09,-9.101838e-08,5.700724e-09,2.327932e-09,-4.713027e-09
